Строим запрос к нашему сервису

In [30]:
import requests

import urllib.request
import json 

In [31]:
# Пример данных
data = (1,"F",40,155,60,81.3,1.2,1,1,1,114,73,94,215,82,73,126,12.9,1,0.7,18,19,27,"Y",0,"Y")

In [32]:
# формируем запрос
def send_json(x):
    id, gender, age, height, weight, waist, eyesight_left, eyesight_right, hearing_left, hearing_right, systolic, relaxation, fasting_blood_sugar, cholesterol, triglyceride, hdl, ldl, hemoglobin, urine_protein, serum_creatinine, ast, alt, gtp, oral, dental_caries, tartar = x
    body = {'ID': int(id),
            'gender': gender,
            'age': int(age),
            'height': int(height),
            'weight': int(weight),
            'waist': waist,
            'eyesight_left': eyesight_left,
            'eyesight_right': eyesight_right,
            'hearing_left': int(hearing_left),
            'hearing_right': int(hearing_right),
            'systolic': int(systolic),
            'relaxation': int(relaxation),
            'fasting_blood_sugar': int(fasting_blood_sugar),
            'cholesterol': int(cholesterol),
            'triglyceride': int(triglyceride),
            'hdl': int(hdl),
            'ldl': int(ldl),
            'hemoglobin': hemoglobin,
            'urine_protein': int(urine_protein),
            'serum_creatinine': serum_creatinine,
            'ast': int(ast),
            'alt': int(alt),
            'gtp': int(gtp),
            'oral': oral,
            'dental_caries': int(dental_caries),
            'tartar': tartar}
    myurl = 'http://7fa2-35-236-155-186.ngrok.io' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [33]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(data)
print('предсказание', response)

предсказание 0.05


Сделаем обработку массы запросов:

1. загрузим данные X_test

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


X_test = pd.read_csv("/content/drive/MyDrive/X_test.csv")
y_test = pd.read_csv("/content/drive/MyDrive/y_test.csv")

2. Передаем запрос одиночный из наших данных и посмотрим на ответ

In [36]:
response = send_json(X_test[['ID', 'gender', 'age', 'height', 'weight', 'waist', 'eyesight_left', 'eyesight_right', 'hearing_left', 'hearing_right', 'systolic', 'relaxation', 'fasting_blood_sugar', 'cholesterol', 'triglyceride', 'hdl', 'ldl', 'hemoglobin', 'urine_protein', 'serum_creatinine', 'ast', 'alt', 'gtp', 'oral', 'dental_caries', 'tartar']].iloc[0,:])

In [37]:
response

0.44

In [38]:
X_test[['ID', 'gender', 'age', 'height', 'weight', 'waist', 'eyesight_left', 'eyesight_right', 'hearing_left', 'hearing_right', 'systolic', 'relaxation', 'fasting_blood_sugar', 'cholesterol', 'triglyceride', 'hdl', 'ldl', 'hemoglobin', 'urine_protein', 'serum_creatinine', 'ast', 'alt', 'gtp', 'oral', 'dental_caries', 'tartar']].head()

,ID,gender,age,height,weight,waist,eyesight_left,eyesight_right,hearing_left,hearing_right,...,ldl,hemoglobin,urine_protein,serum_creatinine,ast,alt,gtp,oral,dental_caries,tartar
0,44185,M,60,155,55,77.0,1.0,1.0,1.0,1.0,...,136.0,14.7,1.0,0.9,24.0,14.0,11.0,Y,0,Y
1,27703,M,40,175,85,90.7,1.2,1.0,1.0,1.0,...,132.0,16.9,1.0,1.2,43.0,64.0,139.0,Y,0,Y
2,40647,M,25,170,80,94.0,1.5,1.5,1.0,1.0,...,215.0,14.6,1.0,0.9,35.0,48.0,30.0,Y,0,N
3,40821,M,60,170,70,88.0,0.5,0.5,1.0,1.0,...,40.0,14.8,1.0,1.0,30.0,45.0,71.0,Y,0,N
4,47062,M,40,165,70,86.2,1.2,1.2,1.0,1.0,...,131.0,16.5,1.0,1.1,24.0,38.0,102.0,Y,1,Y


3. Сделаем N запросов и оценим время

In [39]:
N = 50

In [40]:
%%time
predictions = X_test[
                     ['ID', 'gender', 'age', 'height', 'weight', 'waist', 'eyesight_left', 'eyesight_right', 'hearing_left', 'hearing_right', 'systolic', 'relaxation', 'fasting_blood_sugar', 'cholesterol', 'triglyceride', 'hdl', 'ldl', 'hemoglobin', 'urine_protein', 'serum_creatinine', 'ast', 'alt', 'gtp', 'oral', 'dental_caries', 'tartar']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1)

CPU times: user 296 ms, sys: 28.3 ms, total: 324 ms
Wall time: 19.4 s


In [41]:
predictions.values[:5]

array([0.44, 0.78, 0.38, 0.65, 0.73])

4. Посчитаем метрику

In [42]:
y_test.head()

,smoking
0,0
1,1
2,0
3,0
4,0


In [43]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.4, F-Score=0.783, Precision=0.667, Recall=0.947


In [44]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.8573853989813243

5. Анализ : в целом похоже